# Confiabilidade de viga mista

## Importando bibliotecas

In [23]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

## Definindo funções

In [24]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    amostra: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [25]:
espessura_laje: float = 10
desvpad_espessura_laje: float = 0.06 * espessura_laje

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [26]:
limite_escoamento_perfil: float = 250 * 1.08
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [27]:
limite_escoamento_armadura: float = 500 * 1.08
desvpad_escoamento_armadura: float = 0.08 * limite_escoamento_armadura

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [28]:
resistencia_concreto: float = 20 * 1.17
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [29]:
diametro_rebite: float = 1.59
desvpad_diametro_rebite: float = 0.01 * diametro_rebite

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [30]:
resistencia_rebite: float = 1.59
desvpad_resistencia_rebite: float = 0.01 * resistencia_rebite

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [31]:
carga_permanente_g1: float = 7.6
carga_permanente_g2: float = 5

variacao_carga: list = [5, 9]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9
espacamento_laje: float = 2.8

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * vao_laje ** 2) / 8 for i in range(len(carga_distribuida_projeto))]

altura_laje_ht: float = 45
coeficiente_ya1: float = 1.1
x: float = 10

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_laje_ht / 2) + espessuras_laje[j] - (x / 2))))
    index_matriz += 1
    
print(areas_minimas)


[[19.441605403720924, 18.016384016880977, 14.940428334930129, 15.588257292279582, 13.901528168722903, 18.92450768126925, 16.57781146848921, 15.870259358913692, 17.044489057532473, 17.158523098569532, 15.904966481981946, 18.35084077555019, 15.00033327442145, 15.44551212114349, 14.504389538932418, 17.491579194425036, 14.782459930493848, 14.738452932671079, 17.40117609338615, 16.512407057732695, 14.621631191184603, 16.529482127329075, 15.099824829064705, 17.50230625381688, 15.801765855622442, 18.533294779724027, 16.654598346355197, 15.837058640924456, 18.486539678786706, 17.40593412265411, 17.87723558966623, 14.671649051288759, 17.723195204400827, 18.12741627609759, 15.516508789614933, 14.101610441689017, 15.090985459088769, 15.931259635006489, 18.303121642286996, 17.115779434088648, 16.276526771758576, 16.879033716978956, 17.35170092173866, 17.46489125564607, 15.760456842429134, 15.76576213850275, 16.197058445301444, 15.731130084346155, 16.172906059658256, 14.2684994976195, 17.8771902442